In [1]:
# Importing libraries
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.axes3d import Axes3D
from PIL import Image
import functions
from GP_action import GP_action
import time
import imageio
import pickle

In [2]:
Noise=False# Specify if noisy or not
Noise_level= 0.05 # if noisy, what is the noise sd
'''If the scaled X and Y is being used rememeber to scale the noise as well'''

'If the scaled X and Y is being used rememeber to scale the noise as well'

In [3]:
# The target function is assigned, chose 1 from the below

objective=functions.Ackley()
objective=functions.Rosenbrock()
objective=functions.Synthetic()
objective=functions.sin()
objective=functions.sin_2(Noise,Noise_level)
objective=functions.sincos(Noise,Noise_level)
objective=functions.Branin()
objective=functions.Shubert(Noise,Noise_level)
objective=functions.Kean(Noise,Noise_level)
objective=functions.Shubert()
objective=functions.Kean(Noise,Noise_level)
objective=functions.Hartmann_3()
objective=functions.Hartmann_6()
objective=functions.Ackley_6()
objective=functions.Levy_10()
objective=functions.Rastrigin()

print(objective.bounds)
# Extracting the bounded range of the inputs
#func_bounds=bounds=np.array(
#            [item[1] for item in sorted(objective.bounds.items(), key=lambda x: x[0])],dtype=np.float)
func_bounds=np.array(objective.bounds)
print(func_bounds)
#print(objective.func(np.array([0,0,0,0,0,0,0,0,0,0])))


[(-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1)]
[[-1  1]
 [-1  1]
 [-1  1]
 [-1  1]
 [-1  1]
 [-1  1]
 [-1  1]
 [-1  1]
 [-1  1]
 [-1  1]
 [-1  1]


In [4]:
# max budget for finding good point
Budget=300

# Acq_names=['PI','PG','EI','EG','gp_ucb','random','TS','SATS','G_ES','MES','GD']
acquation_function='gp_ucb'
function_bounds=func_bounds
func_name='Rastrigin'
Max= 0 # function max 3.86278, 3.32237

In [5]:

#acq=['GD','gp_ucb','EI','TS']
acq=['GD','TS','GD_5']
Mean_dict = {} 
std_dict={}
k=0
mean=np.zeros(300)
std=np.zeros(300)
seed=np.random.randint(10000, size=5)
while(k<3):
    temp_out=np.zeros(300)
    seed_count=0
    for m in range(0,5):
        samples=[]
        acquation_function=acq[k]
        seeds=seed[seed_count]
        Bo_test=GP_action(objective.func,function_bounds,Noise,Noise_level,acquation_function)
        Bo_test.initiate(seeds,n_random_draws=5)
        seed_count=seed_count+1
        flag = True
        i = 0
        samples.append(Max-np.max(Bo_test.Y))
        while flag:
            if i >= (Budget-1) :
                break
            Bo_test.sample_new_value()
            Regret= Max-np.max(Bo_test.Y)
            samples.append(Regret)
            i += 1
    
        out=np.array(samples)
        temp_out=np.vstack((temp_out, out))
        
    temp_out = np.delete(temp_out,(0), axis=0)
    mean_temp=np.mean(temp_out,axis=0)
    std_temp=0.5*np.sqrt(np.var(temp_out,axis=0)) # Calculating the one sided interval for 90% confidence

    mean=np.vstack((mean,mean_temp))
    std=np.vstack((std,std_temp))
    Mean_dict[acquation_function]=mean_temp
    std_dict[acquation_function]=std_temp
    k=k+1
    


1 1
1
estimated lengthscale and variance [0.83135557 1.07186132]
1 1
2
1 1


KeyboardInterrupt: 

In [ ]:
def save_obj(obj, name ):
    with open('data/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('data/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
#save_obj(Mean_dict,func_name+'_Mean_'+'noisy_'+'Regret')
#save_obj(std_dict,func_name+'_Error_'+'noisy_'+'Regret')
save_obj(Mean_dict,func_name+'_Mean_'+'Regret')
save_obj(std_dict,func_name+'_Error_'+'Regret')



In [ ]:
plt.style.use('default')
import seaborn as sns
colors = sns.color_palette("dark")
greens = sns.color_palette("BuGn", 10)
blues = sns.color_palette("PuBu", 10)
reds = sns.color_palette("YlOrRd", 10)
purples = sns.color_palette("PuOr", 10)

def save_plot_T(mean,std):
    GD=np.arange(0,300,1)
    GP_UCB=np.arange(0,300,1)
    EI=np.arange(0,300,1)
    i=300
    size=30 # error every size
    
    # naming the x axis 
    fig,(ax1)=plt.subplots(figsize=(4.5, 3))
    plt.xlabel('Queries', fontsize=10) 
    # naming the y axis 
    plt.ylabel('Simple Regret',fontsize=10) 
    # giving a title to my graph 
    plt.title('')
    # show a legend on the plot 
    plt.legend() 
    ax1.plot(GD[:i], mean['GD'][:i],'-',color=reds[-1]) 
#    ax1.plot(GP_UCB[:i], mean['gp_ucb'][:i],'--',color=blues[-1]) 
 #   ax1.plot(EI[:i], mean['EI'][:i],'-.',color=purples[-1])
    ax1.plot(EI[:i], mean['TS'][:i],'-.',color=greens[-1])
    ax1.plot(GD[:i], mean['GD_5'][:i],'-',color=blues[-1]) 
    
    ax1.errorbar(GD[:i],mean['GD'][:i],yerr=std['GD'][:i],label="GD",linestyle='-',color=reds[-1],fmt='s',capsize=3,errorevery=size,markevery=size,lw=1.5)
 #   ax1.errorbar(GP_UCB[:i],mean['gp_ucb'][:i],yerr=std['gp_ucb'][:i],label="gp_ucb",linestyle='--',color= blues[-1],fmt='^',capsize=3,errorevery=size,markevery=size,lw=1.5)
 #   ax1.errorbar(EI[:i],mean['EI'][:i],yerr=std['EI'][:i],label="EI",linestyle='-.',color=purples[-1],fmt='>',capsize=3,errorevery=size,markevery=size,lw=1.5)
    ax1.errorbar(EI[:i],mean['TS'][:i],yerr=std['TS'][:i],label="TS",linestyle=':',color=greens[-1],fmt='x',capsize=3,errorevery=size,markevery=size,lw=1.5)
    ax1.errorbar(GD[:i],mean['GD_5'][:i],yerr=std['GD_5'][:i],label="GD_5",linestyle='--',color=blues[-1],fmt='>',capsize=3,errorevery=size,markevery=size,lw=1.5)


    ax1.legend(loc='upper right',fontsize=10)
    # function to show the plot
#    plt.fill_between(x, mean[1]-std[1], mean[1]+std[1],color='C0', alpha = 0.3)
#    plt.fill_between(x, mean[2]-std[2], mean[2]+std[2],color='C2', alpha = 0.3)
#    plt.fill_between(x, mean[3]-std[3], mean[3]+std[3],color='C4', alpha = 0.3)
    
    plt.show()
    fig.savefig('Ackley_noisless.pdf', bbox_inches='tight')

In [ ]:
#mean=load_obj(func_name+'_Mean_'+'noisy_'+'Regret')
#error=load_obj(func_name+'_Error_'+'noisy_'+'Regret')
mean=load_obj(func_name+'_Mean_'+'Regret')
error=load_obj(func_name+'_Error_'+'Regret')
save_plot_T(mean,error)
